In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mars-spec-train/train_labels.csv
/kaggle/input/mars-spec-train/train_features/S0459.csv


### 1. Loading data and libraries 

In [2]:
# for processing numbers and files csv
import pandas as pd
import numpy as np

# for visualization
# import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

# preprocessing 
# from scipy.fft import fft
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# model related
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import tensorflow as tf
# other
import copy
# from scipy.signal import find_peaks


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# A. Data description
###  input data 
<ul> 
<li>there are 1000 training examples</li>
<li>each of them is dataframe with time, mass , and intensity:: sorted by time then mass</li>
<li>dataframe lenghts are variable</li>
</ul>

### output binary clssification
<ul> 
<li>positive class probability</li>
</ul>


# B. Feature Extraction Steps
<ul>
<li>group mass values upto 1 decimal (0.0 to 650.0 , diff = 0.5)           /<br>-- total (1300) or multiply by 2  and take round() </li>
<li>take time values and fill remaining with (0.00 to 53.00 , diff = 0.02) /<br>-- total (2650) or multiply by 50 and take round() </li>
<li>create a Time X Mass matrix (2650,1300) with intensity values</li>

<li>noramlize intensity values --
    <ol> 
    <li>subtract mean</li>
    <li>minmax scale</li>
    </ol>   
</li>
<li>total 2650x1300 values = 3,445,000</li> 
</ul>

# C. Model architecture
<ul>
<li>feed this as convolutional layers (2,5 2d conv stride 2, 5 , that is comvolute these values)  . This will group similar time and mass values</li>
<li>feed this as pooling layers ( pooling layers with mean/ max pooling ) </li>
<li>further data reduction steps</li>
<li>then fully connected layers</li>
<li>sigmoid or softmax layer</li>
<li>output positive label probability</li>
</ul>


In [3]:
metadata = pd.read_csv('/kaggle/input/mars-spec-train/metadata.csv').fillna(0)
train_labels = pd.read_csv('/kaggle/input/mars-spec-train/train_labels.csv')

In [4]:
metadata.head(2)

,sample_id,split,derivatized,features_path,features_md5_hash
0,S0000,train,0.0,train_features/S0000.csv,52ec6d6f8372500ab4e069b5fbdae6f9
1,S0001,train,0.0,train_features/S0001.csv,348f90baed8a8189bf0d4c7b9ed9f965


In [5]:
train_labels.head(2)

,sample_id,aromatic,hydrocarbon,carboxylic_acid,nitrogen_bearing_compound,chlorine_bearing_compound,sulfur_bearing_compound,alcohol,other_oxygen_bearing_compound,mineral
0,S0000,0,0,0,0,0,0,0,0,1
1,S0001,0,0,0,0,0,0,0,0,0


### 2. Preprocessing, Feature extraction

In [6]:
# constants
num_mass_values = 1350
num_time_values = 2650
matrix_shape = (num_time_values, num_mass_values)


In [7]:

# Function to create the Time X Mass matrix for a single data sample
def create_matrix(id ):
    
    sample_id, split, derivate, path, _ = metadata.iloc[id]
#     print(path)
    path = '/kaggle/input/mars-spec-train/'+ path
    data = pd.read_csv(path)

    label = None
    if (split == 'train'):
        label = train_labels.iloc[id].values[1:]

    # discretization of time and mass
    data.time = (data.time*50).round()
    data.mass = (data.mass*2).round()
    data.time = data.time.astype(int)
    data.mass = data.mass.astype(int)
    
    # scaling intensity
    data.intensity = MinMaxScaler().fit_transform(data.intensity.values.reshape(-1, 1))
    
    # filling matrix
    matrix = np.zeros(matrix_shape)
    matrix[data['time'],data['mass']] = data['intensity']

    return matrix , label


In [8]:
sample_matrix, sample_label= create_matrix(np.random.randint(809)) ## 0.2 second # max = 1120
sample_matrix[0:100, 100:200],sample_label
# print(sample_matrix[500:1000,np.random.randint(0,100)])

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object))

Dummy model

In [9]:
training_data = range(809)  # Load your training data


In [10]:
class DataGenerator(Sequence):
    
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
    def __len__(self):
        return len(self.data) // self.batch_size

    def __getitem__(self, idx):        
        
        low = idx * self.batch_size
        high = min(low + self.batch_size, len(self.data))
        batch_x = []
        batch_y = []

        for i,data_sample in enumerate(self.data[low:high]):
            processed_sample, processed_label = self.preprocess_data(data_sample)
            batch_x.append(processed_sample)
            batch_y.append(processed_label)
        return np.array(batch_x), np.array(batch_y)

    def preprocess_data(self, data_sample):
        # Perform the feature extraction steps and create the matrix
        matrix, label = create_matrix(data_sample)
        if type(label) == np.ndarray:
            label = np.array(label).astype(int)
        return matrix , label
    
    def getLabel(self, data_sample):
        matrix, label = create_matrix(data_sample)
        if type(label) == np.ndarray:
            label = np.array(label).astype(int)
        return label


In [11]:
train_data, val_data = train_test_split(training_data, test_size=0.2, random_state=42)
train_labels.iloc[[3,334,534]].values[:,1:].astype(int)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
def log_loss_agg(y_test,y_pred):
    y_test = y_test.astype(np.float16)
    y_pred = y_pred.astype(np.float16)
    N,M = y_test.shape
    a=[]
    for m in range(M):
        loss=0
        for i in range(N):
            loss -= ((y_test[i,m]*np.log(y_pred[i,m]))+((1.0-y_test[i,m])*np.log(1.0-y_pred[i,m])))
        loss = loss/N
        a.append(round(loss,8))
    return np.array(a)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import log_loss
model = Sequential()

model.add(Conv2D(4, kernel_size=(5, 5),strides = (2,1), activation='relu', input_shape=(2650, 1350, 1)))
model.add(MaxPooling2D(pool_size=(80, 10), strides=(40,5)))

model.add(Flatten())

model.add(Dense(32, activation='relu'))

model.add(Dense(9, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "AUC"])

# Define the batch size and number of epochs
batch_size = 16
num_epochs = 32

# Create instances of the custom data generator for training and validation
train_data_generator = DataGenerator(train_data, batch_size)
val_data_generator = DataGenerator(val_data, batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1323, 1346, 4)     104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 268, 4)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 34304)             0         
                                                                 
 dense (Dense)               (None, 32)                1097760   
                                                                 
 dense_1 (Dense)             (None, 9)                 297       
                                                                 
Total params: 1,098,161
Trainable params: 1,098,161
Non-trainable params: 0
______________________________________________

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', patience = 3 )
history = model.fit(train_data_generator, epochs=num_epochs, validation_data=val_data_generator, callbacks=[es])

Epoch 1/32
40/40 [==============================] - 533s 13s/step - loss: 0.3626 - accuracy: 0.2391 - auc: 0.6745 - val_loss: 0.2472 - val_accuracy: 0.3688 - val_auc: 0.8242
Epoch 2/32
40/40 [==============================] - 519s 13s/step - loss: 0.2493 - accuracy: 0.3781 - auc: 0.8352 - val_loss: 0.2184 - val_accuracy: 0.3938 - val_auc: 0.8764
Epoch 3/32
40/40 [==============================] - 533s 13s/step - loss: 0.2127 - accuracy: 0.4141 - auc: 0.8878 - val_loss: 0.1913 - val_accuracy: 0.3812 - val_auc: 0.9142
Epoch 4/32
40/40 [==============================] - 565s 14s/step - loss: 0.1869 - accuracy: 0.4500 - auc: 0.9170 - val_loss: 0.1733 - val_accuracy: 0.4375 - val_auc: 0.9299
Epoch 5/32
40/40 [==============================] - 569s 14s/step - loss: 0.1629 - accuracy: 0.4578 - auc: 0.9436 - val_loss: 0.1621 - val_accuracy: 0.4500 - val_auc: 0.9396
Epoch 6/32
40/40 [==============================] - 574s 14s/step - loss: 0.1451 - accuracy: 0.4906 - auc: 0.9580 - val_loss: 0.16

In [15]:
# model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save('model3.keras')
# metadata.to_csv("opopo")

In [16]:
train_loss = 0
train_lab = []
train_pred = []
from sklearn.metrics import log_loss
for i,data in enumerate(train_data):
    mat , lab = create_matrix(data)
    lab = np.array(lab).astype(int).reshape(1,9)
    pred = model.predict(mat.reshape(-1,2650,1350,1), verbose = 0)
    train_lab.append(lab)
    train_pred.append(pred)
    if (i%100 == 0):
        print (i)

0
100
200
300
400
500
600


In [17]:

train_lab = np.array(train_lab).reshape(-1,9)
train_pred = np.array(train_pred).reshape(-1,9)
print(f"train log loss : {log_loss(train_lab, train_pred)}")

train log loss : 0.9294082760524709


In [18]:
val_loss = 0
val_lab = []
val_pred = []
from sklearn.metrics import log_loss
for i,data in enumerate(val_data):
    mat , lab = create_matrix(data)
    lab = np.array(lab).astype(int).reshape(1,9)
    pred = model.predict(mat.reshape(-1,2650,1350,1), verbose = 0)
    val_lab.append(lab)
    val_pred.append(pred)
    if (i%100 == 0):
        print (i)

0
100


In [19]:
val_lab = np.array(val_lab).reshape(-1,9)
val_pred = np.array(val_pred).reshape(-1,9)

In [20]:
print(f"valid log loss : {log_loss(val_lab, val_pred).mean()}")

valid log loss : 1.0541745896278631


train log loss : 1.0316461173431106 \
valid log loss : 1.0560311513842007 \
sample predictions below

In [21]:
for i in range (10):
    sample_matrix, sample_label= create_matrix(np.random.randint(809)) ## 0.2 second # max = 1120
    print (f"{i+1}th prediction")
    print("prediction : ",(model.predict(sample_matrix.reshape(-1, 2650,1350,1),verbose = 0).round().astype(int)).squeeze())
    print("true label : ",np.array(sample_label))

1th prediction
prediction :  [1 0 1 1 0 0 0 0 0]
true label :  [1 0 1 1 0 0 0 0 0]
2th prediction
prediction :  [0 1 0 0 0 0 0 0 0]
true label :  [0 1 0 0 0 0 0 0 0]
3th prediction
prediction :  [0 0 0 0 0 0 0 0 0]
true label :  [0 0 0 0 0 0 1 0 0]
4th prediction
prediction :  [0 0 0 0 0 0 0 0 0]
true label :  [0 0 0 0 0 0 0 0 0]
5th prediction
prediction :  [0 0 0 0 0 0 0 0 0]
true label :  [0 0 0 0 0 0 0 0 0]
6th prediction
prediction :  [0 0 0 0 0 0 0 0 1]
true label :  [0 0 0 0 0 0 0 0 0]
7th prediction
prediction :  [0 0 0 0 0 0 0 0 1]
true label :  [0 0 0 0 0 0 0 0 1]
8th prediction
prediction :  [0 0 0 0 0 0 0 0 0]
true label :  [0 0 0 0 0 0 0 0 0]
9th prediction
prediction :  [0 0 0 0 0 0 0 0 0]
true label :  [0 0 0 0 0 0 0 0 0]
10th prediction
prediction :  [0 1 0 0 0 0 0 0 0]
true label :  [0 1 0 0 0 0 0 0 0]
